In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,2020-12-27T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1027,174,1201,31966,...,2298,NaN,NaN,89616,1188827,793985.0,NaN,14.0,NaN,NaN
6464,2020-12-27T17:00:00,ITA,9,Toscana,43.769231,11.255889,847,165,1012,9788,...,3588,NaN,NaN,118784,1849528,1050691.0,NaN,4.0,Positivi diagnosticati solo con test antigenic...,NaN
6465,2020-12-27T17:00:00,ITA,10,Umbria,43.106758,12.388247,260,36,296,3257,...,604,NaN,NaN,28064,492558,249041.0,NaN,5.0,NaN,NaN
6466,2020-12-27T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,83,2,85,347,...,374,NaN,NaN,7181,61613,37039.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,2020-12-27T17:00:00,19,Sicilia,1027,174,1201,31966,33167,-123,682,54151,2298,NaN,NaN,89616,1188827,793985.0,14.0,NaN,NaN
6464,2020-12-27T17:00:00,9,Toscana,847,165,1012,9788,10800,-78,227,104396,3588,NaN,NaN,118784,1849528,1050691.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
6465,2020-12-27T17:00:00,10,Umbria,260,36,296,3257,3553,4,36,23907,604,NaN,NaN,28064,492558,249041.0,5.0,NaN,NaN
6466,2020-12-27T17:00:00,2,Valle d'Aosta,83,2,85,347,432,14,27,6375,374,NaN,NaN,7181,61613,37039.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-27', '2020-12-26')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-27T17:00:00,1,Piemonte,3039,211,3250,27812,31062,-495,470,155416,7784,NaN,NaN,194262,1654351,974905.0,14.0,NaN,NaN
1,2020-12-27T17:00:00,2,Valle d'Aosta,83,2,85,347,432,14,27,6375,374,NaN,NaN,7181,61613,37039.0,0.0,NaN,NaN
2,2020-12-27T17:00:00,3,Lombardia,3801,508,4309,55018,59327,-295,466,387761,24867,NaN,NaN,471955,4785738,2585071.0,16.0,NaN,NaN
3,2020-12-27T17:00:00,5,Veneto,2626,360,2986,87035,90021,1179,3337,144593,6038,NaN,NaN,240652,3232475,1196750.0,21.0,NaN,NaN
4,2020-12-27T17:00:00,6,Friuli Venezia Giulia,626,58,684,11506,12190,65,144,33913,1564,NaN,NaN,47667,907700,335221.0,4.0,NaN,NaN
5,2020-12-27T17:00:00,7,Liguria,695,66,761,5008,5769,97,99,50489,2846,NaN,NaN,59104,695223,318446.0,5.0,I 146664 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-27T17:00:00,8,Emilia-Romagna,2609,210,2819,55356,58175,888,1283,100659,7493,NaN,NaN,166327,2508298,1210120.0,9.0,NaN,NaN
7,2020-12-27T17:00:00,9,Toscana,847,165,1012,9788,10800,-78,227,104396,3588,NaN,NaN,118784,1849528,1050691.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-27T17:00:00,10,Umbria,260,36,296,3257,3553,4,36,23907,604,NaN,NaN,28064,492558,249041.0,5.0,NaN,NaN
9,2020-12-27T17:00:00,11,Marche,428,65,493,9178,9671,36,181,28499,1527,NaN,NaN,39697,514332,305197.0,3.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-26T17:00:00,1,Piemonte,3011,208,3219,28338,31557,60,417,154468,7767,NaN,NaN,193792,1651049,973322.0,1.0,NaN,NaN
1,2020-12-26T17:00:00,2,Valle d'Aosta,77,4,81,337,418,-10,24,6364,372,NaN,NaN,7154,61248,36914.0,1.0,NaN,NaN
2,2020-12-26T17:00:00,3,Lombardia,3839,513,4352,55270,59622,912,1606,387049,24818,NaN,NaN,471489,4780837,2583057.0,14.0,NaN,NaN
3,2020-12-26T17:00:00,5,Veneto,2589,355,2944,85898,88842,1457,2523,142487,5986,NaN,NaN,237315,3219624,1192654.0,28.0,NaN,NaN
4,2020-12-26T17:00:00,6,Friuli Venezia Giulia,587,56,643,11482,12125,41,224,33849,1549,NaN,NaN,47523,906413,334858.0,5.0,NaN,NaN
5,2020-12-26T17:00:00,7,Liguria,672,66,738,4934,5672,3,34,50489,2844,NaN,NaN,59005,694040,318045.0,2.0,NaN,NaN
6,2020-12-26T17:00:00,8,Emilia-Romagna,2630,214,2844,54443,57287,-1693,1756,100333,7424,NaN,NaN,165044,2502232,1208257.0,19.0,NaN,NaN
7,2020-12-26T17:00:00,9,Toscana,825,166,991,9887,10878,-391,402,104099,3580,NaN,NaN,118557,1846026,1049142.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-26T17:00:00,10,Umbria,248,34,282,3267,3549,3,51,23878,601,NaN,NaN,28028,491673,248707.0,2.0,NaN,NaN
9,2020-12-26T17:00:00,11,Marche,412,64,476,9159,9635,68,206,28359,1522,NaN,NaN,39516,513159,304656.0,6.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-27T17:00:00,1,Piemonte,3039,211,3250,27812,31062,-495,470,155416,7784,NaN,NaN,194262,1654351,974905.0,14.0,NaN,NaN
1,2020-12-27T17:00:00,2,Valle d'Aosta,83,2,85,347,432,14,27,6375,374,NaN,NaN,7181,61613,37039.0,0.0,NaN,NaN
2,2020-12-27T17:00:00,3,Lombardia,3801,508,4309,55018,59327,-295,466,387761,24867,NaN,NaN,471955,4785738,2585071.0,16.0,NaN,NaN
3,2020-12-27T17:00:00,4,Trentino-Alto Adige,510,65,575,11780,12355,63,143,35875,1622,0.0,0.0,49852,797749,310917.0,4.0,0,NaN
4,2020-12-27T17:00:00,5,Veneto,2626,360,2986,87035,90021,1179,3337,144593,6038,NaN,NaN,240652,3232475,1196750.0,21.0,NaN,NaN
5,2020-12-27T17:00:00,6,Friuli Venezia Giulia,626,58,684,11506,12190,65,144,33913,1564,NaN,NaN,47667,907700,335221.0,4.0,NaN,NaN
6,2020-12-27T17:00:00,7,Liguria,695,66,761,5008,5769,97,99,50489,2846,NaN,NaN,59104,695223,318446.0,5.0,I 146664 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-27T17:00:00,8,Emilia-Romagna,2609,210,2819,55356,58175,888,1283,100659,7493,NaN,NaN,166327,2508298,1210120.0,9.0,NaN,NaN
8,2020-12-27T17:00:00,9,Toscana,847,165,1012,9788,10800,-78,227,104396,3588,NaN,NaN,118784,1849528,1050691.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-27T17:00:00,10,Umbria,260,36,296,3257,3553,4,36,23907,604,NaN,NaN,28064,492558,249041.0,5.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-26T17:00:00,1,Piemonte,3011,208,3219,28338,31557,60,...,154468,7767,NaN,NaN,193792,1651049,973322.0,1.0,NaN,NaN
1,1,2020-12-26T17:00:00,2,Valle d'Aosta,77,4,81,337,418,-10,...,6364,372,NaN,NaN,7154,61248,36914.0,1.0,NaN,NaN
2,2,2020-12-26T17:00:00,3,Lombardia,3839,513,4352,55270,59622,912,...,387049,24818,NaN,NaN,471489,4780837,2583057.0,14.0,NaN,NaN
3,19,2020-12-26T17:00:00,4,Trentino-Alto Adige,486,65,551,11741,12292,-49,...,35802,1615,0.0,0.0,49709,796373,310580.0,2.0,0,NaN
4,3,2020-12-26T17:00:00,5,Veneto,2589,355,2944,85898,88842,1457,...,142487,5986,NaN,NaN,237315,3219624,1192654.0,28.0,NaN,NaN
5,4,2020-12-26T17:00:00,6,Friuli Venezia Giulia,587,56,643,11482,12125,41,...,33849,1549,NaN,NaN,47523,906413,334858.0,5.0,NaN,NaN
6,5,2020-12-26T17:00:00,7,Liguria,672,66,738,4934,5672,3,...,50489,2844,NaN,NaN,59005,694040,318045.0,2.0,NaN,NaN
7,6,2020-12-26T17:00:00,8,Emilia-Romagna,2630,214,2844,54443,57287,-1693,...,100333,7424,NaN,NaN,165044,2502232,1208257.0,19.0,NaN,NaN
8,7,2020-12-26T17:00:00,9,Toscana,825,166,991,9887,10878,-391,...,104099,3580,NaN,NaN,118557,1846026,1049142.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-26T17:00:00,10,Umbria,248,34,282,3267,3549,3,...,23878,601,NaN,NaN,28028,491673,248707.0,2.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-27T17:00:00,1,Piemonte,3039,211,3250,27812,31062,-495,470,...,7784,NaN,NaN,194262,1654351,974905.0,14.0,NaN,NaN,4341375
1,2020-12-27T17:00:00,2,Valle d'Aosta,83,2,85,347,432,14,27,...,374,NaN,NaN,7181,61613,37039.0,0.0,NaN,NaN,125501
2,2020-12-27T17:00:00,3,Lombardia,3801,508,4309,55018,59327,-295,466,...,24867,NaN,NaN,471955,4785738,2585071.0,16.0,NaN,NaN,10103969
3,2020-12-27T17:00:00,4,Trentino-Alto Adige,510,65,575,11780,12355,63,143,...,1622,0.0,0.0,49852,797749,310917.0,4.0,0,NaN,1074819
4,2020-12-27T17:00:00,5,Veneto,2626,360,2986,87035,90021,1179,3337,...,6038,NaN,NaN,240652,3232475,1196750.0,21.0,NaN,NaN,4907704
5,2020-12-27T17:00:00,6,Friuli Venezia Giulia,626,58,684,11506,12190,65,144,...,1564,NaN,NaN,47667,907700,335221.0,4.0,NaN,NaN,1211357
6,2020-12-27T17:00:00,7,Liguria,695,66,761,5008,5769,97,99,...,2846,NaN,NaN,59104,695223,318446.0,5.0,I 146664 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-27T17:00:00,8,Emilia-Romagna,2609,210,2819,55356,58175,888,1283,...,7493,NaN,NaN,166327,2508298,1210120.0,9.0,NaN,NaN,4467118
8,2020-12-27T17:00:00,9,Toscana,847,165,1012,9788,10800,-78,227,...,3588,NaN,NaN,118784,1849528,1050691.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-27T17:00:00,10,Umbria,260,36,296,3257,3553,4,36,...,604,NaN,NaN,28064,492558,249041.0,5.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-27T17:00:00,1,Piemonte,3039,211,3250,27812,31062,-495,470,...,NaN,NaN,194262,1654351,974905.0,14.0,NaN,NaN,4341375,575
1,2020-12-27T17:00:00,2,Valle d'Aosta,83,2,85,347,432,14,27,...,NaN,NaN,7181,61613,37039.0,0.0,NaN,NaN,125501,20
2,2020-12-27T17:00:00,3,Lombardia,3801,508,4309,55018,59327,-295,466,...,NaN,NaN,471955,4785738,2585071.0,16.0,NaN,NaN,10103969,1036
3,2020-12-27T17:00:00,4,Trentino-Alto Adige,510,65,575,11780,12355,63,143,...,0.0,0.0,49852,797749,310917.0,4.0,0,NaN,1074819,106
4,2020-12-27T17:00:00,5,Veneto,2626,360,2986,87035,90021,1179,3337,...,NaN,NaN,240652,3232475,1196750.0,21.0,NaN,NaN,4907704,1016
5,2020-12-27T17:00:00,6,Friuli Venezia Giulia,626,58,684,11506,12190,65,144,...,NaN,NaN,47667,907700,335221.0,4.0,NaN,NaN,1211357,180
6,2020-12-27T17:00:00,7,Liguria,695,66,761,5008,5769,97,99,...,NaN,NaN,59104,695223,318446.0,5.0,I 146664 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-27T17:00:00,8,Emilia-Romagna,2609,210,2819,55356,58175,888,1283,...,NaN,NaN,166327,2508298,1210120.0,9.0,NaN,NaN,4467118,634
8,2020-12-27T17:00:00,9,Toscana,847,165,1012,9788,10800,-78,227,...,NaN,NaN,118784,1849528,1050691.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-27T17:00:00,10,Umbria,260,36,296,3257,3553,4,36,...,NaN,NaN,28064,492558,249041.0,5.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-27T17:00:00,5,Veneto,2626,360,2986,87035,90021,1179,3337,...,35.43,1137,814,12851,4096.0,52,2106,25.97,0.06800,4.90
1,2020-12-27T17:00:00,8,Emilia-Romagna,2609,210,2819,55356,58175,888,1283,...,33.12,913,-473,6066,1863.0,69,326,21.15,0.02872,3.72
2,2020-12-27T17:00:00,12,Lazio,2802,301,3103,73335,76438,144,977,...,35.54,80,-146,9815,3702.0,16,817,9.95,0.01666,2.69
3,2020-12-27T17:00:00,19,Sicilia,1027,174,1201,31966,33167,-123,682,...,29.59,-140,345,5630,3444.0,15,790,12.11,0.01373,1.80
4,2020-12-27T17:00:00,1,Piemonte,3039,211,3250,27812,31062,-495,470,...,36.70,-526,53,3302,1583.0,17,948,14.23,0.01083,4.47
5,2020-12-27T17:00:00,3,Lombardia,3801,508,4309,55018,59327,-295,466,...,49.03,-252,-1140,4901,2014.0,49,712,9.51,0.00461,4.67
6,2020-12-27T17:00:00,15,Campania,1440,95,1535,78540,80075,-488,310,...,18.81,-494,-229,3382,974.0,8,790,9.17,0.00536,3.22
7,2020-12-27T17:00:00,9,Toscana,847,165,1012,9788,10800,-78,227,...,31.55,-99,-175,3502,1549.0,8,297,6.48,0.00610,3.19
8,2020-12-27T17:00:00,16,Puglia,1444,154,1598,52040,53638,-152,221,...,41.73,-157,-323,1748,1544.0,12,361,12.64,0.00551,2.16
9,2020-12-27T17:00:00,11,Marche,428,65,493,9178,9671,36,181,...,45.45,19,-25,1173,541.0,5,140,15.43,0.01192,2.61


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,65,41,106,61.32
1,Lombardia,508,528,1036,49.03
2,Marche,65,78,143,45.45
3,Puglia,154,215,369,41.73
4,Piemonte,211,364,575,36.70
5,Lazio,301,546,847,35.54
6,Veneto,360,656,1016,35.43
7,Emilia-Romagna,210,424,634,33.12
8,Friuli Venezia Giulia,58,122,180,32.22
9,Liguria,66,143,209,31.58


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2020-12-25T17:00:00,22,P.A. Trento,306,49,355,1422,1777,31,154,18170,892,NaN,NaN,20839,440570,149543.0,2.0,NaN,"Dei 154 casi odierni, 23 sono molecolari con p..."
612,2020-12-26T17:00:00,21,P.A. Bolzano,155,17,172,10394,10566,2,24,17462,718,NaN,NaN,28746,354359,160672.0,0.0,NaN,NaN
613,2020-12-26T17:00:00,22,P.A. Trento,331,48,379,1347,1726,-51,124,18340,897,NaN,NaN,20963,442014,149908.0,2.0,NaN,"Dei 124 casi odierni, 56 sono molecolari con p..."
614,2020-12-27T17:00:00,21,P.A. Bolzano,154,20,174,10406,10580,14,57,17503,720,NaN,NaN,28803,355149,160905.0,3.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2020-12-23T17:00:00,4,Trentino-Alto Adige,516,66,582,10298,10880,-478,288,...,11.0,5587.0,1112.0,-448.0,11.34,5.35,5.15,25.90,-8.33,-4.21
304,2020-12-24T17:00:00,4,Trentino-Alto Adige,483,66,549,10304,10853,-27,380,...,14.0,4451.0,905.0,6.0,12.02,5.06,8.54,41.99,0.00,-0.25
305,2020-12-25T17:00:00,4,Trentino-Alto Adige,453,68,521,11820,12341,1488,336,...,12.0,5001.0,1748.0,1516.0,13.05,4.22,6.72,19.22,3.03,13.71
306,2020-12-26T17:00:00,4,Trentino-Alto Adige,486,65,551,11741,12292,-49,148,...,12.0,1723.0,-500.0,-79.0,11.80,4.48,8.59,-29.60,-4.41,-0.40


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
